# Introduction

Search or information retrieval, to use it's more general technical term, is something all of use almost all the time. In any tech stack i have built / worked in in the past decade or so, search has played some role. Typically we use Lucene / Elasticsearch or one of it's technical clones or variants. About a week or so ago i came across some papers on **Neural Information Retrieval**. It led me on some interesting fishing expeditions last couple of weekends and the output of these is, a toy project, that compares classical IR techniques with neural IR.

## Classical IR

Some background about classical Information Retrieval (IR). Of course there are whole textbooks on this field, but in case you are not fully familiar this section will cover some of the basics so that it is easier to understand this toy project and it's results. Search problem can be simply stated as:

**Given a corpus of documents, and a query, the problem of search can be simply stated as finding the best match(es).**

Naively one can simply say, if i am looking for *Chennai* simply grep and return all documents in the corpus that contains the word *Chennai*. Of course, needless to say this would be a very naive thing to do. There are whole conferences and disserations dedicated to what "best match" means, how to define it, how to quantify it, how to calculate it efficiently at scale etc. So, how does a search system return results ? By scoring each document in a corpus documents for a given input query. The scoring function f(q,d) is a real valued function that takes into account all query terms and all document terms and outputs a score. Scores are then ranked across the corpus and best matches are identified. 

One of the popular scoring function is Okapi BM25 (BM stands for Best Matching) or BM25 for short. Okapi BM25 is based on the probabilistic retrieval framework. Probabilistic Relevance Model makes an estimation of the probability of finding if a document dj is relevant to a query q. Even though BM25 is motivated by a probabilistic interpretation, I find it easier to understand BM25 as a bag-of-words retrieval function that ranks a set of documents based on the query terms appearing in each document, regardless of the inter-relationship between the query terms within a document (e.g., their relative proximity). 

\begin{equation*}
Score(D,Q) = \sum_{i=1}^n IDF(q_i) \frac {(f(q_i,D)(k_1 + 1))} {(f(q_i,D) + k_1) (1 - b + b \frac {|D|} {avgdl})}
\end{equation*}

where f(q_i,D) is q_i's term frequency in the document D, |D| is the length of the document D in words, and avgdl is the average document length in the text collection from which documents are drawn, k_1 and b are free parameters.

\begin{equation*}
IDF(q_i) = \log \frac {N - n(q_i) + 0.5} {n(q_i) + 0.5}
\end{equation*}

where N is the total number of documents in the collection, and n(q_i) is the number of documents containing q_i.

So how are query terms and documents represented ? Vector space model or term vector model is an algebraic model for representing text documents and queries as vectors of identifiers. Each dimension corresponds to a separate term. If a term occurs in the document, its value in the vector is non-zero. There are several different ways of computing these values, one of the best known schemes is tf-idf weighting. Here tf–idf, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. The tf-idf value increases proportionally to the number of times a word appears in the document, but is often offset by the frequency of the word in the corpus, which helps to adjust for the fact that some words appear more frequently in general. TF stands for Term Frequency and IDF stands for Inverse Document Frequency. For example, if am searching for *Chennai*, a document containing the word *Chennai* many times must score higher that a document that contains the word fewer times. However it is also true that number of occurences should also have some sort of diminishing marginal utility. Presence of second occurence of the word *Chennai* in the document has more importance than say 100th occurence of the same word in the document. IDF is used to offset effect of words that occur too frequently in any language ("the" "a" etc.) IDF attempts to capture the idea that the specificity of a term can be quantified as an inverse function of the number of documents in which it occurs. So IDF will value rare terms higher than commonly occuring terms.

## Neural IR

Traditional IR models use local representations of terms for query-document matching. However as most of us can immediately guess, inspecting non-query terms in the document for garnering evidence of relevance can be useful in many cases. For example if a model can learn that *Madras* was the previous name for the same city now called *Chennai* it can be much more efficient in search in some cases. So looking at other terms in the document to infer that this document is about the same city is not possible in traditional term counting based IR approaches.

An embedding is a representation of items in a new space such that the properties of, and the relationships between, the items are preserved. The goal of an embedding is to generate a simpler representation. Neural term embedding models are typically trained by setting up a prediction task. Both the term and the features have one-hot representations in the input and the output layers, respectively, and the model learns dense low-dimensional representations in the process of minimizing the prediction error.

Word2vec is one such popular embedding. For word2vec, the features for a term are made up of its neighbours within a fixed size window over the text from the training corpus. The skip-gram architecture is a simple one hidden layer neural network. The model has two different weight matrices Win and Wout that are learnable parameters of the models. Win gives us the IN embeddings corresponding to all the input terms and Wout corresponding to the OUT embeddings for the output terms. Generally, only Win is used and Wout is discarded after training, but we will see later that our toy project makes use of both the IN and the OUT embeddings.

### Dual Embedding Space Model (DESM)

Mitra et al. [https://arxiv.org/pdf/1602.01137.pdf] point out that when using word2vec embeddings for IR it is more appropriate to represent the query terms using the IN embeddings and the document terms using the OUT embeddings of the trained model. In this Dual Embedding Space Model (DESM) the word2vec embeddings are trained on search
queries, which empirically performs better than training on document body text. Training on short queries, however, makes the inter-term similarity more pronouncedly Typical (where, “Yale” is closer to “Harvard” and “NYU”) when both terms are represented using their IN vectors—better retrieval performance is achieved instead by using the IN-OUT similarity (where, “Yale” is closer to “faculty” and “alumni”) that mirrors more the Topical notions of relatedness


# Goal

* This toy project is basically a comparison of BM25 and DESM on a small corpus.
* We use input data from https://www.kaggle.com/snapcrack/all-the-news. The data primarily falls between the years of 2016 and July 2017, although there is a not-insignificant number of articles from 2015, and a possibly insignificant number from before then. It is all news articles, a total of **142573** short articles from New York Times, Breitbart, CNN, Business Insider, the Atlantic, Fox News, Talking Points Memo, Buzzfeed News, National Review, New York Post, the Guardian, NPR, Reuters, Vox, and the Washington Post. Sampling isn't quite scientific. It is like a random noisy corpus.
* Each row of CSV contains:
 * an id for the Sqlite database
 * author name
 * full date
 * month
 * year
 * title
 * publication name
 * article url (not available for all articles)
 * full article content


## Tools

* We use **gensim** an open source python library for all our analysis.

## Data Prep

* We extract just the article from csvs (just the *content* column).
* We lower case and use NLTK sentence and word tokenizers.
* Once lower cased, it has 519186 unique words. 
* We name the documents as 1.txt, 2.txt etc. Given how gensim corpora handles document identification this becomes easier to map results back.

## BM25 Setup

* We first construct a dictionary and corpus using **gensim**.
* We then use it's BM25 scoring functions.
* Note that documents are identified by the order in which they are added to the corpora. So numbered input files come in handy.

## DESM Setup

* We use **gensim** word2vec implementation to generate word2vec embeddings.
* Word2Vec training on 592108745 raw words (444871464 effective words) took 5432.0s, 81898 effective words/s on my macbook pro.
* Embeddings produces are 100 dimensional.
* In the word2vec model, there are two linear transforms that take a word in vocab space to a hidden layer (the "in" vector), and then back to the vocab space (the "out" vector). Usually this out vector is discarded after training.  However when we do `model.save` using **gensim** it saves both input and output weights. The input and output embeddings are syn0 and syn1 (or syn1neg, for negative sampling), respectively.


In [10]:
import sys, os, csv, glob, json, uuid, pickle, math
import nltk 
import gensim, logging
import numpy as np, scipy, pandas as pd
from operator import itemgetter

In [7]:
CONTENT_INDEX = 9
csv.field_size_limit(sys.maxsize)
CONTENT_PATH = './inputs/contents/'
TOKENS_PATH = './inputs/tokens/'
CENTROIDS_PATH = './inputs/centroids/'
BM25_PATH = './inputs/bm25/'

if not os.path.exists(CONTENT_PATH):
    os.makedirs(CONTENT_PATH)
    
if not os.path.exists(TOKENS_PATH):
    os.makedirs(TOKENS_PATH)
    
if not os.path.exists(CENTROIDS_PATH):
    os.makedirs(CENTROIDS_PATH)

if not os.path.exists(BM25_PATH):
    os.makedirs(BM25_PATH)

Data Prep stage.

In [12]:
count = 0

for fname in glob.iglob('./inputs/*.csv', recursive=False):
    f = open(fname)
    reader = csv.reader(f)
    for line in reader:
        count = count + 1
        content = line[CONTENT_INDEX]
        cname = CONTENT_PATH + str(count) + '.txt'
        tname = TOKENS_PATH + str(count) + '.tokens'
        cf = open(cname, 'w')
        cf.write(content)
        cf.close()
        tf = open(tname, 'w')
        for sentence in nltk.sent_tokenize(content):
            tf.write("%s\n" % sentence.lower())
        tf.close()

We want do not want to load all documents into memory. Even for this small corpus it is over 140K documents. We create a lazy python iterator that word2vec model uses to grab data.

In [2]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in glob.iglob(self.dirname +'*.tokens', recursive=True):
            for line in open(fname):
                yield nltk.word_tokenize(line)

In [3]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = MySentences('./inputs/tokens/') 
model1 = gensim.models.Word2Vec(sentences, min_count=1)

2017-09-15 10:47:03,294 : INFO : collecting all words and their counts
2017-09-15 10:47:03,520 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-09-15 10:47:06,035 : INFO : PROGRESS: at sentence #10000, processed 222985 words, keeping 20486 word types
2017-09-15 10:47:08,676 : INFO : PROGRESS: at sentence #20000, processed 453071 words, keeping 30146 word types
2017-09-15 10:47:11,281 : INFO : PROGRESS: at sentence #30000, processed 676789 words, keeping 37469 word types
2017-09-15 10:47:13,805 : INFO : PROGRESS: at sentence #40000, processed 901606 words, keeping 43566 word types
2017-09-15 10:47:16,458 : INFO : PROGRESS: at sentence #50000, processed 1124248 words, keeping 48895 word types
2017-09-15 10:47:19,159 : INFO : PROGRESS: at sentence #60000, processed 1348471 words, keeping 53519 word types
2017-09-15 10:47:21,885 : INFO : PROGRESS: at sentence #70000, processed 1580287 words, keeping 57840 word types
2017-09-15 10:47:24,544 : INFO : PROGRESS: 

Save the trained model. In the word2vec model, there are two linear transforms that take a word in vocab space to a hidden layer (the "in" vector), and then back to the vocab space (the "out" vector). Usually this out vector is discarded after training.  However when we do `model.save` using **gensim** it saves both input and output weights. The input and output embeddings are syn0 and syn1 (or syn1neg, for negative sampling), respectively.

In [4]:
model1.save('./model/w2v-lc.model')
model1.wv.save_word2vec_format('./model/w2v-lc.model.bin', binary=True)
vocab = dict([(k, v.index) for k, v in model1.wv.vocab.items()])
with open('./model/w2v-lc-vocab.json', 'w') as f:
    f.write(json.dumps(vocab))

2017-09-15 12:41:09,508 : INFO : saving Word2Vec object under ./model/w2v-lc.model, separately None
2017-09-15 12:41:09,510 : INFO : storing np array 'syn0' to ./model/w2v-lc.model.wv.syn0.npy
2017-09-15 12:41:09,828 : INFO : not storing attribute syn0norm
2017-09-15 12:41:09,830 : INFO : storing np array 'syn1neg' to ./model/w2v-lc.model.syn1neg.npy
2017-09-15 12:41:10,129 : INFO : not storing attribute cum_table
2017-09-15 12:41:11,950 : INFO : saved ./model/w2v-lc.model
2017-09-15 12:41:11,952 : INFO : storing 519186x100 projection weights into ./model/w2v-lc.model.bin


Do a simple test. It is interesting that it thinks texas : senate = alabama : congress. Not a bad start to have learnt some context.

In [5]:
model1.wv.most_similar(positive=['texas', 'senate'], negative=['alabama'])

2017-09-15 13:12:35,701 : INFO : precomputing L2-norms of word weight vectors


[('congress', 0.6408563852310181),
 ('legislature', 0.6165897250175476),
 ('senate’s', 0.595465362071991),
 ('congressional', 0.5766005516052246),
 ('filibuster', 0.5701662302017212),
 ('senators', 0.5530926585197449),
 ('democrats', 0.5497519373893738),
 ('gop', 0.5479837656021118),
 ('repeal', 0.5465051531791687),
 ('republicans', 0.5443344116210938)]

In [ ]:
for fname in glob.iglob('./inputs/contents/*.txt', recursive=False):
    for line in open(fname):
        centroid_in = (np.mean(np.array([get_embedding(x) for x in nltk.word_tokenize(line.lower())]), axis=0))
        centroid_out = (np.mean(np.array([get_embedding(x, out=True) for x in nltk.word_tokenize(line.lower())]), axis=0))
        out_dict = { fname : (centroid_in, centroid_out) }
        pickle_file = './inputs/centroids/' + os.path.basename(fname).replace('.txt', '.p')
        pickle.dump(out_dict, open(pickle_file, "wb"))

Next we get on to BM25. First we build the dictionary, followed by corpus.

In [6]:
class BM25Sentences(object):
    def __init__(self, pattern):
        self.pattern = pattern
 
    def __iter__(self):
        for fname in glob.iglob(self.pattern, recursive=True):
            for line in open(fname):
                yield nltk.word_tokenize(line)

In [ ]:
sentences = BM25Sentences('./inputs/tokens/*.tokens')
dictionary = gensim.corpora.Dictionary(line for line in sentences)
dictionary.compactify()
print(dictionary)

In [ ]:
dictionary.save('./inputs/bm25/allnews.dict')

In [ ]:
bm25dict = dictionary.load('./inputs/bm25/allnews.dict') 

class MyCorpus(object):
    def __init__(self, dirname):
        self.dirname = dirname
        self.count = 142573
 
    def __iter__(self):
        for x in range(self.count):
            fname = self.dirname + str(x+1) + '.tokens'
            doc = open(fname).read().replace('\n', '')
            yield bm25dict.doc2bow(nltk.word_tokenize(doc))

In [ ]:
citer = MyCorpus(TOKENS_PATH)
corpus = [x for x in citer]
print (len(corpus))

In [ ]:
gensim.corpora.MmCorpus.serialize('./inputs/bm25/allnewscorpus.mm', corpus)

In [ ]:
bm25corpus = gensim.corpora.MmCorpus('./inputs/bm25/allnewscorpus.mm')

# Testing

We test both DESM and BM25 with the same input query. The query we choose is kinda long (by query standards) and is rather generic. It is **political stability and economic health**. We stick to top 5 results.

## DESM Test

* As mentioned in the DESM paper, we compute the centroid of documents and compute the cosine similarity of the query words with document centroid.
* We present the results with both DESM-IN-OUT and DESM-IN-IN metrics (see paper for details)

## BM25 Test

* We compute BM25 scores across the corpus for the input query and pick the top 5.

In [8]:
model = gensim.models.Word2Vec.load('./model/w2v-lc.model')

2017-09-16 21:51:30,499 : INFO : loading Word2Vec object from ./model/w2v-lc.model
2017-09-16 21:51:33,776 : INFO : loading wv recursively from ./model/w2v-lc.model.wv.* with mmap=None
2017-09-16 21:51:33,777 : INFO : loading syn0 from ./model/w2v-lc.model.wv.syn0.npy with mmap=None
2017-09-16 21:51:34,085 : INFO : setting ignored attribute syn0norm to None
2017-09-16 21:51:34,087 : INFO : loading syn1neg from ./model/w2v-lc.model.syn1neg.npy with mmap=None
2017-09-16 21:51:34,452 : INFO : setting ignored attribute cum_table to None
2017-09-16 21:51:34,453 : INFO : loaded ./model/w2v-lc.model


In [11]:
centroid_dict = {}
for fname in glob.iglob('./inputs/centroids/*.p', recursive=False):
    centroid_dict.update(pickle.load(open(fname, "rb")))

In [12]:
clean_centroid_dict = {k: centroid_dict[k] for k in centroid_dict if not np.isnan(centroid_dict[k][0]).any()}

In [13]:
def get_embedding(x, out=False):
    if x in model.wv.vocab:
        if out == True:
            return model.syn1neg[model.wv.vocab[x].index]
        else:
            return model[x]
    else:
        return np.zeros(100)

In [14]:
def score_document(q_embeddings, d_centroid):
    individual_csims = [(1 - scipy.spatial.distance.cosine(qin, d_centroid)) for qin in q_embeddings]
    return (sum(individual_csims)/len(q_embeddings))

In [23]:
bm25dict = gensim.corpora.Dictionary().load('./inputs/bm25/allnews.dict') 
bm25corpus = gensim.corpora.MmCorpus('./inputs/bm25/allnewscorpus.mm')
bm25 = gensim.summarization.bm25.BM25(bm25corpus)
average_idf = sum(float(val) for val in bm25.idf.values()) / len(bm25.idf)

2017-09-16 22:04:23,261 : INFO : loading Dictionary object from ./inputs/bm25/allnews.dict
2017-09-16 22:04:23,722 : INFO : loaded ./inputs/bm25/allnews.dict
2017-09-16 22:04:23,754 : INFO : loaded corpus index from ./inputs/bm25/allnewscorpus.mm.index
2017-09-16 22:04:23,756 : INFO : initializing corpus reader from ./inputs/bm25/allnewscorpus.mm
2017-09-16 22:04:23,760 : INFO : accepted corpus with 142573 documents, 519186 features, 46972445 non-zero entries


In [26]:
query = 'political stability and economic health'
query_words = nltk.word_tokenize(query.lower())

In [27]:
scores = bm25.get_scores(bm25dict.doc2bow(query_words), average_idf)

In [35]:
best_result = ['./inputs/contents/'+str(x+1)+'.txt' for x in (sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:5])]
for fname in best_result:
    print(fname)

./inputs/contents/30931.txt
./inputs/contents/40023.txt
./inputs/contents/71852.txt
./inputs/contents/133532.txt
./inputs/contents/1620.txt


In [30]:
query_ins = [get_embedding(x) for x in query_words]
q_len = len(query_ins)
print('Num words in query: ', len(query_words), 'Num query word in vectors: ', q_len)

Num words in query:  5 Num query word in vectors:  5


In [31]:
scores_in_in = []
scores_in_out = []
for k,v in clean_centroid_dict.items():
    scores_in_in.append((k, score_document(query_ins, v[0])))
    scores_in_out.append((k, score_document(query_ins, v[1])))

scores_in_in = sorted(scores_in_in, key=itemgetter(1), reverse=True)
scores_in_out = sorted(scores_in_out, key=itemgetter(1), reverse=True)

In [32]:
print('TOP 5 IN-IN:')
top_5_in_in = [x[0] for x in scores_in_in[:5]]

for fname in top_5_in_in:
    print(fname)

TOP 5 IN-IN:
./inputs/contents/140797.txt
./inputs/contents/32221.txt
./inputs/contents/31472.txt
./inputs/contents/39594.txt
./inputs/contents/135444.txt


In [33]:
print('TOP 5 IN-OUT:')
top_5_in_out = [x[0] for x in scores_in_out[:5]]

for fname in top_5_in_out:
    print(fname)

TOP 5 IN-OUT:
./inputs/contents/73280.txt
./inputs/contents/140797.txt
./inputs/contents/32221.txt
./inputs/contents/42404.txt
./inputs/contents/42105.txt


# Conclusion

The documents returned by DESM are clearly much higher quality. They certainly are documents related to **political stability and economic health**. However documents returned by BM25 are solely based on term matchings.

It is quite impressive how word2vec trained on the corpus has learnt word contexts. I was very skeptical about taking the centroid of w2v representations (as mentioned in DESM paper) (as someone who is used to dealing with loads of geo spatial data where i routinely hit centroid outside of the object in question, but i digress) i was wondering how taking the centroid would capture all that the doc has to say. It works remarkably well. Also to some extent all my content is news articles. Typically they tend to be focussed on a single topic, instead of meandering prose.

One could also argue fairly that my query is biased. It is sort of vague and is not specific like looking for **Mango**. Obviously word2vec is designed to excel at it as compared to BM25, which is based on just bag of words with no contextual information.